In [2]:
# Set root directory
import os

ROOT_DIR = "D:\Coding\CZ4010\Applied-Cryptography-Project"
os.chdir(ROOT_DIR)

In [4]:
from utils import ascii_to_binary_list, binary_list_to_ascii
from LWE_PKC import LWE_Encrypt, LWE_Decrypt
from math import comb

### 1. Testing out the Cryptosystem

In [5]:
bit_message = [1]

In [6]:
# Weak Parameters
n = 3
q = 17
max_error = 1
S = 2*max_error + 1   # Cardinality of the error set
m = 2*comb(n+S, S)    # The number of eqations must be >> (n+S)C(s)

In [7]:
# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

# Get public keys
A_list, T_list, q, max_error = lwe_d.get_public_keys()
# print(A_list)
# print(T_list)
print(f"Randomly Initalized Secret: {lwe_d.secret}")

Randomly Initalized Secret: [9, 2, 2]


In [8]:
# Encrypt Message
lwe_e = LWE_Encrypt(A_list, T_list, q, max_error)
A_new, T_send = lwe_e.encrypt_message(bit_message)
print(f"A_new = {A_new}\nT_send = {T_send}")

A_new = [array([11, 15, 11], dtype=int32)]
T_send = [8]


In [9]:
# Decrypt Message
decrypted_binary_message = lwe_d.decrypt_message(A_new, T_send)
decrypted_message = binary_list_to_ascii(decrypted_binary_message)
print(f"Decrypted Message = {decrypted_binary_message}")

Decrypted Message = [1]


### Arora-Ge Attack

A brute force attack when the max_error parameter is small.

In [732]:
# Weak Parameters
n = 1
q = 11
max_error = 1
E = 2*max_error + 1   # Cardinality of the error set
m = int(20 * comb(n+E, E))    # The number of eqations must be >> (n+S)C(s)

In [733]:
# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

# The secret is randomly initalized in the class
A_list, b_list, q, max_error = lwe_d.get_public_keys()
print(f"Randomly Initalized Secret: {lwe_d.secret}")

Randomly Initalized Secret: [5]


We are going to recover the secret `s` from `A_list` and `b_list` given the following weak parameters,

1. The error is truncated and takes values in the set $\{-1, 0, 1\} \implies |E| = 3$ 
<br>
<br>
2. The number of equations in `A_list`, $m >> {n + |E| \choose |E|}$

In [734]:
print(f"Cardinality of error set: {E}")
print(f"Number of equations: {len(A_list)} >> {comb(n+E, E)}")

Cardinality of error set: 3
Number of equations: 80 >> 4


In [735]:
from itertools import product
from collections import defaultdict
from sympy import symbols, Eq, Mod, Poly, Matrix, GF
from sympy.polys.matrices import DomainMatrix

In [736]:
# Define the secret vector
secret_vector = symbols(f'x1:{n+1}')
secret_vector

(x1,)

In [737]:
polynomials_over_Zq = []
error_set = [i for i in range(-max_error, max_error+1)] 

# Begin constructing the polynomials for each LWE instance <A, b>
for A, b in zip(A_list, b_list):
    # Initalize the polynomial term to the identity polynomial of the finite field
    polynomial_over_Zq = GF(q)[secret_vector](1)

    for e in error_set:
        # Multiply each variable by its corresponding weight
        weighted_polynomial = sum(w * var for w, var in zip(A, secret_vector))
        # print(weighted_polynomial)

        # Construct the weighted polynomial (this is the AT*s term in the equation)
        weighted_secret_polynomial_over_Zq = GF(q)[secret_vector](weighted_polynomial)
        # print(weighted_secret_polynomial_over_Zq)

        # Complete the term (b - AT*s - e)
        term = b - weighted_secret_polynomial_over_Zq - e
        # print(term)

        # Accumulate the product of each term over Zq
        polynomial_over_Zq = polynomial_over_Zq * term
        # print(polynomial_over_Zq)

    #print(polynomial_over_Zq)
    polynomials_over_Zq.append(polynomial_over_Zq)

print(len(polynomials_over_Zq))

80


In [738]:
print(polynomials_over_Zq[0])

9 mod 11*x1**3 + 8 mod 11*x1**2 + 6 mod 11


In [739]:
polynomials_over_Zq[1].terms()

[((3,), SymmetricModularIntegerMod11(5)), ((2,), SymmetricModularIntegerMod11(7)), ((1,), SymmetricModularIntegerMod11(1)), ((0,), SymmetricModularIntegerMod11(9))]

In [740]:
def generate_tuples(n, S):
    # Use itertools.product to generate all tuples
    tuples = list(product(range(S + 1), repeat=n))
    return tuples

tuples = generate_tuples(n, S)

In [741]:
coefficients_dicts = []

for polynomial in polynomials_over_Zq:
    coefficients_dict = defaultdict(int, {key: 0 for key in tuples})

    for term_key, coeff in polynomial.terms():
        coefficients_dict[term_key] = int(coeff) # NOTE: Converting to negative int!!!

    coefficients_dicts.append(coefficients_dict)

In [742]:
coefficients_dicts[0]

defaultdict(<class 'int'>, {(0,): -5, (1,): 0, (2,): -3, (3,): -2})

In [743]:
row_order = []

# Let's keep the degree 1 terms up front for convenince
for i in range(n):
    term = [0]*n
    term[i] = 1
    row_order.append(tuple(term))

constant_term_key = tuple([0]*n)
seen = set(row_order)
candidates = generate_tuples(n, S)

for x in candidates:
    if x not in seen and x != constant_term_key:
        row_order.append(x)

row_order.append(constant_term_key)

# Sanoty check
assert len(row_order) == len(generate_tuples(n, S))

In [744]:
coefficient_matrix = []
rhs = []

for coeff_dict in coefficients_dicts:
    row = []

    for key in row_order[:-1]:
        row.append(coeff_dict[key])
    
    # Append row
    coefficient_matrix.append(row)

    # Append the rhs
    negative_constant_term = -coeff_dict[row_order[-1]]
    rhs.append(negative_constant_term)

In [745]:
# Solving linear system using DomainMatrix
m = Matrix(coefficient_matrix)
b = Matrix(rhs)

# Convert matrices to finite field of order q (q is prime):
K = GF(q, symmetric=False)
dm = DomainMatrix.from_Matrix(m).convert_to(K)
bm = DomainMatrix.from_Matrix(b).convert_to(K)

# Print shape of system of equations
print(m.shape)
print(b.shape)

# Solve and convert back to an ordinary Matrix:
solution_vector = dm.lu_solve(bm).to_Matrix()
print(solution_vector)

(80, 3)
(80, 1)
Matrix([[5], [3], [4]])


In [746]:
# import numpy as np

# Ax = np.array(coefficient_matrix)
# d = np.array(rhs)

# # Print shape of system of equations
# print(Ax.shape)
# print(d.shape)

# # Get the dimensions of the original matrix
# num_rows, num_cols = Ax.shape

# # Determine the size of padding needed
# padding_size = max(0, num_rows - num_cols)

# # Pad the matrix with zeros to make it square
# padded_matrix = np.pad(Ax, ((0, 0), (0, padding_size)), mode='constant')

# # Print shape of the padded matrix
# print("Padded Matrix Shape:", padded_matrix.shape)

# try:
#     solution_base_10 = np.linalg.solve(padded_matrix, d)
# except:
#     solution_base_10 = np.linalg.lstsq(padded_matrix, d)

# print(solution_base_10)

(80, 3)
(80,)
Padded Matrix Shape: (80, 80)
(array([ 0.04512065, -0.28698019,  0.05304032,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,

C:\Users\rahul\AppData\Local\Temp\ipykernel_27352\1197516898.py:25: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  solution_base_10 = np.linalg.lstsq(padded_matrix, d)


In [747]:
print(f"Randomly Initalized Secret: \t\t\t{lwe_d.secret}")
print(f"Secret obtained from Arora-Ge Attack: \t{secret_vector} = {solution_vector[:n]}")

Randomly Initalized Secret: 			[5]
Secret obtained from Arora-Ge Attack: 	(x1,) = [5]
